In [1]:
import geemap
import ee
import os
import pandas as pd

In [2]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
os.chdir(r'E:/Work/Sample_points/')
roi_path = r'latitude.xls'
roi_df = pd.read_excel(roi_path)

In [4]:
points = geemap.df_to_ee(roi_df)
Map.addLayer(points, {}, 'points')

In [5]:
def sample_points_to_df(img, ps, col_names=None):
    ps_data = img.sampleRegions(ps, 
                                geometries=True, 
                                scale=img.projection().nominalScale())
    values_list = [f['properties'] for f in ps_data.getInfo()["features"]]
    if col_names == None:
        values_df = pd.DataFrame(values_list)
    else:
        values_df = pd.DataFrame(values_list)[col_names]
    return values_df

In [ ]:
images = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")
columns = ["ID", "longitude", "latitude"] + images.first().bandNames().getInfo()
for year in [2019, 2021]:
    image = images.filterDate(str(year), str(year+1)).mean()
    
    df = sample_points_to_df(image, points, columns)
    
    df.to_csv(f'updata_GLDAS_{year}.csv', index=False, header=True)
    print(f'updata_GLDAS_{year}.csv output succeeded!!')

In [ ]:
images = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")
columns = ["ID", "longitude", "latitude"] + images.first().bandNames().getInfo()
for year in [2019, 2021]:
    images_date = images.filterDate(str(year), str(year+1))
    image = images_date.mean()
    prcp = images_date.select('pr').sum()
    
    df = sample_points_to_df(image, points, columns)
    df_pr = sample_points_to_df(prcp, points)
    
    df['pr'] = df_pr['pr']
    df.to_csv(f'updata_IDAHO_EPSCOR_{year}.csv', index=False, header=True)
    print(f'updata_IDAHO_EPSCOR_{year}.csv output succeeded!!')

In [8]:
images = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
columns = ["ID", "longitude", "latitude"] + images.first().bandNames().getInfo()
for year in [2021]:
    images_date = images.filterDate(str(year), str(year+1))
    image = images_date.mean()
    prcp = images_date.select('total_precipitation').sum()
    
    df = sample_points_to_df(image, points, columns)
    df_pr = sample_points_to_df(prcp, points)
    
    df['total_precipitation'] = df_pr['total_precipitation']
    df.to_csv(f'updata_ERA5_{year}.csv', index=False, header=True)
    print(f'updata_ERA5_{year}.csv output succeeded!!')

updata_ERA5_2021.csv output succeeded!!
